In [ ]:
!git clone https://github.com/NVIDIA/CleanUNet.git


Cloning into 'CleanUNet'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 46 (delta 14), reused 31 (delta 12), pack-reused 4
Receiving objects: 100% (46/46), 28.19 KiB | 1.28 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Filtering content: 100% (2/2), 351.59 MiB | 36.59 MiB/s, done.


In [ ]:
%cd CleanUNet

/content/CleanUNet


In [ ]:
import torch
import os
from network import CleanUNet

model_path = '/content/CleanUNet/exp/DNS-large-full/checkpoint/pretrained.pkl'

model = CleanUNet()
# model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CleanUNet(
  (encoder): ModuleList(
    (0): Sequential(
      (0): Conv1d(1, 64, kernel_size=(4,), stride=(2,))
      (1): ReLU()
      (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (1): Sequential(
      (0): Conv1d(64, 128, kernel_size=(4,), stride=(2,))
      (1): ReLU()
      (2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (2): Sequential(
      (0): Conv1d(128, 256, kernel_size=(4,), stride=(2,))
      (1): ReLU()
      (2): Conv1d(256, 512, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (3): Sequential(
      (0): Conv1d(256, 512, kernel_size=(4,), stride=(2,))
      (1): ReLU()
      (2): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (4): Sequential(
      (0): Conv1d(512, 768, kernel_size=(4,), stride=(2,))
      (1): ReLU()
      (2): Conv1d(768, 1536, kernel_size=(1,), stride=(1,))
      (3): GLU(dim=1)
    )
    (5-7): 3 x Sequential(
      (0): Conv

In [ ]:
import torchaudio
import torch

clean_waveform, sample_rate = torchaudio.load('/content/source.wav')
noise_waveform, _ = torchaudio.load('/content/noise.wav')
noise_waveform = noise_waveform[:, :clean_waveform.size(1)]
noise_level = 0.5  # می‌توانید مقدار این متغیر را تغییر دهید
noisy_waveform = clean_waveform + noise_level * noise_waveform
torchaudio.save('noisy_audio.wav', noisy_waveform, sample_rate)

In [ ]:
from google.colab import files

files.download('noisy_audio.wav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch
import torchaudio

input_audio_path = '/content/noisy_audio.wav'
output_audio_path = '/content/cleaned_audio.wav'
waveform, sample_rate = torchaudio.load(input_audio_path)

waveform = waveform.to(device)

with torch.no_grad():
    cleaned_waveform = model(waveform)

In [ ]:
import torchaudio
import torch
import numpy as np
import IPython.display as ipd
from scipy.io.wavfile import write

if cleaned_waveform.dim() == 1:
    cleaned_waveform = cleaned_waveform.unsqueeze(0)

cleaned_waveform = cleaned_waveform.cpu().numpy()
write('cleaned_audio.wav', sample_rate, cleaned_waveform.T)
files.download('cleaned_audio.wav')
ipd.Audio('cleaned_audio.wav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>